In [101]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, expr, lit, max, sum, udf
from pyspark.sql.types import IntegerType, StringType, NumericType, StructField, StructType
from pyspark.sql import Row
from pyspark.storagelevel import StorageLevel

In [2]:
spark = (SparkSession.builder
            .master("local[*]")
            .appName("Spark Documentation")
            .getOrCreate())

23/07/17 18:44:00 WARN Utils: Your hostname, MacBook-Air-de-Gabriel.local resolves to a loopback address: 127.0.0.1; using 192.168.1.8 instead (on interface en0)
23/07/17 18:44:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/17 18:44:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Documentation topic order
- SparkSession
- SQLContext
- UDFRegistration
- DataFrame
- GroupedData
- Column
- Catalog
- Row
- DataFrameNaFunctions
- DataFrameStatFunctions
- DataFrameReader
- DataFrameWriter
- CoGroupedData

- Types
- Functions
- avro.Functions
- Streaming


In [24]:
df_customers = spark.read.csv("../dataset/csv_data/olist_customers_dataset.csv", inferSchema=True, header=True)
df_orders = spark.read.csv("../dataset/csv_data/olist_orders_dataset.csv", inferSchema=True, header=True)
df_order_items = spark.read.csv("../dataset/csv_data/olist_order_items_dataset.csv", inferSchema=True, header=True)

In [198]:
df_customers.cache()
df_orders.cache()
df_order_items.cache()

23/07/15 16:35:54 WARN CacheManager: Asked to cache already cached data.


DataFrame[order_id: string, order_item_id: int, product_id: string, seller_id: string, shipping_limit_date: timestamp, price: double, freight_value: double]

In [136]:
df_order_items.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [88]:
# Select columns in Dataframe
df_order_items.price
df_order_items['price']
col('price')
expr('price')

df_order_items.select(max(df_order_items.price).alias('total_price')).show()
df_order_items.select(max(df_order_items['price']).alias('total_price')).show()
df_order_items.select(max(col('price')).alias('total_price')).show()
df_order_items.select(max('price').alias('total_price')).show()
df_order_items.select(expr('max(price)').alias('total_price')).show()

+-----------+
|total_price|
+-----------+
|     6735.0|
+-----------+

+-----------+
|total_price|
+-----------+
|     6735.0|
+-----------+

+-----------+
|total_price|
+-----------+
|     6735.0|
+-----------+

+-----------+
|total_price|
+-----------+
|     6735.0|
+-----------+

+-----------+
|total_price|
+-----------+
|     6735.0|
+-----------+



In [129]:
# agg, groupBy, alias
df_order_items.groupBy('product_id').agg({'price': 'sum', 'freight_value': 'sum'}).show(2)
df_order_items.groupBy('product_id').agg(sum('price').alias('total_price'), sum('freight_value').alias('total_freight_value')).show(2)
df_order_items.groupBy('product_id').agg(expr('min(price)').alias('min_price')).show(2) # With expr, it's not necessary import functions
df_order_items.groupBy(df_order_items.product_id).agg(expr('min(price)').alias('min_price')).show(2)
df_order_items.agg(expr('count(*)').alias('qnt_rows')).show(2)
df_order_items.groupBy('product_id').min('price').show(2) # This min method is from GroupedData class

+--------------------+----------+------------------+
|          product_id|sum(price)|sum(freight_value)|
+--------------------+----------+------------------+
|0b0172eb0fd18479d...|    404.28|            269.31|
|42a2bd596fda1baef...|    1927.9|            364.31|
+--------------------+----------+------------------+
only showing top 2 rows

+--------------------+-----------+-------------------+
|          product_id|total_price|total_freight_value|
+--------------------+-----------+-------------------+
|0b0172eb0fd18479d...|     404.28|             269.31|
|42a2bd596fda1baef...|     1927.9|             364.31|
+--------------------+-----------+-------------------+
only showing top 2 rows

+--------------------+---------+
|          product_id|min_price|
+--------------------+---------+
|0b0172eb0fd18479d...|    19.89|
|42a2bd596fda1baef...|     79.9|
+--------------------+---------+
only showing top 2 rows

+--------------------+----------+
|          product_id|min(price)|
+----------

In [137]:
# GroupedData.pivot
df_customers.groupBy().pivot('customer_state').agg(count('*')).show()

+---+---+---+---+----+----+----+----+----+---+-----+---+---+---+---+----+---+----+-----+---+---+---+----+----+---+-----+---+
| AC| AL| AM| AP|  BA|  CE|  DF|  ES|  GO| MA|   MG| MS| MT| PA| PB|  PE| PI|  PR|   RJ| RN| RO| RR|  RS|  SC| SE|   SP| TO|
+---+---+---+---+----+----+----+----+----+---+-----+---+---+---+---+----+---+----+-----+---+---+---+----+----+---+-----+---+
| 81|413|148| 68|3380|1336|2140|2033|2020|747|11635|715|907|975|536|1652|495|5045|12852|485|253| 46|5466|3637|350|41746|280|
+---+---+---+---+----+----+----+----+----+---+-----+---+---+---+---+----+---+----+-----+---+---+---+----+----+---+-----+---+



In [92]:
(df_order_items
     .select('order_id', 'product_id', 'price')
     .groupBy('product_id')
     .agg(expr('min(price)').alias('min_price'))
     .show(2))

(df_order_items
     .select('order_id', 'product_id', 'price')
     .groupBy('product_id')
     .agg(expr('min(price)').alias('min_price'))
     .select('min_price')
     .show(2))

+--------------------+---------+
|          product_id|min_price|
+--------------------+---------+
|0b0172eb0fd18479d...|    19.89|
|42a2bd596fda1baef...|     79.9|
+--------------------+---------+
only showing top 2 rows

+---------+
|min_price|
+---------+
|    19.89|
|     79.9|
+---------+
only showing top 2 rows



In [123]:
# expr inside select
(df_customers
     .where(col('customer_state').isin('SC', 'SP'))
     .select('customer_state', expr('CASE WHEN customer_state = "SP" THEN TRUE ELSE FALSE END').alias('state_cat'))
     .groupBy('customer_state', 'state_cat')
     .agg(count("*").alias('count'))
     .show())

# expr using selectExpr
(df_customers
     .where(col('customer_state').isin('SC', 'SP'))
     .selectExpr('customer_state', 'CASE WHEN customer_state = "SP" THEN 1 ELSE 0 END AS state_cat')
     .groupBy('customer_state', 'state_cat')
     .agg(count("*").alias('count'))
     .show())

# expr inside groupBy
(df_customers
     .where(col('customer_state').isin('SC', 'SP'))
     .groupBy('customer_state', expr('CASE WHEN customer_state = "SP" THEN 1 ELSE 0 END').alias('state_cat'))
     .agg(count("*").alias('count'))
     .show())

+--------------+---------+-----+
|customer_state|state_cat|count|
+--------------+---------+-----+
|            SC|    false| 3637|
|            SP|     true|41746|
+--------------+---------+-----+

+--------------+---------+-----+
|customer_state|state_cat|count|
+--------------+---------+-----+
|            SC|        0| 3637|
|            SP|        1|41746|
+--------------+---------+-----+

+--------------+---------+-----+
|customer_state|state_cat|count|
+--------------+---------+-----+
|            SC|        0| 3637|
|            SP|        1|41746|
+--------------+---------+-----+



In [126]:
(df_order_items
     .groupBy('product_id')
     .sum('price')
     .show(2))

+--------------------+----------+
|          product_id|sum(price)|
+--------------------+----------+
|0b0172eb0fd18479d...|    404.28|
|42a2bd596fda1baef...|    1927.9|
+--------------------+----------+
only showing top 2 rows



In [124]:
# withColumn
df_customers.withColumn('novo_zip', expr('customer_zip_code_prefix + 1')).show(2)
df_customers.withColumn('novo_zip', col('customer_zip_code_prefix') + 1).show(2)

+--------------------+--------------------+------------------------+--------------------+--------------+--------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|novo_zip|
+--------------------+--------------------+------------------------+--------------------+--------------+--------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|   14410|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|    9791|
+--------------------+--------------------+------------------------+--------------------+--------------+--------+
only showing top 2 rows

+--------------------+--------------------+------------------------+--------------------+--------------+--------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|novo_zip|
+--------------------+--------------------+--------------------

In [168]:
# UDF register
f = lambda x: x**2

# Register from a python function
spark.udf.register('py_square', udf_square)
spark.sql('SELECT py_square(3) AS square').show()

# Register from an UDF
udf_square = udf(f, IntegerType())
spark.udf.register('udf_square', udf_square)
spark.sql('SELECT udf_square(4) AS square').show()

+------+
|square|
+------+
|     9|
+------+

+------+
|square|
+------+
|    16|
+------+



In [196]:
# UDF
f = lambda x: x**2
udf_square = udf(f, IntegerType()).asNondeterministic()
df_customers.select(udf_square(lit(4)).alias('integer_lit')).distinct().show()

# UDF function
def func(x):
    return x**2

udf_square = udf(func, IntegerType())
df_customers.select(udf_square(lit(5)).alias('integer_lit')).distinct().show()

# UDF annotation
@udf
def func(x):
    return x**2

df_customers.select(func(lit(6)).alias('integer_lit')).distinct().show()

@udf(returnType=StringType())
def func(x):
    return x**2

df_customers.select(func(lit(7)).alias('integer_lit')).distinct().show()

+-----------+
|integer_lit|
+-----------+
|         16|
+-----------+

+-----------+
|integer_lit|
+-----------+
|         25|
+-----------+

+-----------+
|integer_lit|
+-----------+
|         36|
+-----------+

+-----------+
|integer_lit|
+-----------+
|         49|
+-----------+



In [208]:
# Coalesce, Repartition

df_customers.rdd.getNumPartitions()
df_customers.coalesce(2).rdd.getNumPartitions()
df_customers.repartition(4).rdd.getNumPartitions()

4

In [232]:
# Temp View - Live during Spark Application
df_customers.createOrReplaceGlobalTempView('view_temp_customers')
spark.sql("SELECT * FROM global_temp.view_temp_customers").show(2)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 2 rows



In [14]:
# createDataFrame
schema = StructType([
    StructField('language', StringType(), False),
    StructField('users_count', IntegerType(), False)
])
columns = ["language", "users_count"]
data_list = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]
data_row = [Row("Java", 20000), Row("Python", 100000), Row("Scala", 3000)]
data_dict = [{'name': 'Alice', 'age': 1}, {'name': 'Gabriel', 'age': 34}]
data_dict_nested = [{'name': 'Alice', 'age': 1}, {'name': 'Gabriel', 'age': 34}]

# Create from RDD with toDF
rdd = spark.sparkContext.parallelize(data_list)
rdd.toDF(columns).show()
# Create from RDD with createDataFrame and column name
spark.createDataFrame(rdd).toDF(*columns).show()
# Create from RDD with createDataFrame and column name
spark.createDataFrame(rdd, schema).show()
# Create from list with createDataFrame and column name
spark.createDataFrame(data_list).toDF(*columns).show()
# Create from list
spark.createDataFrame(data_list, columns).show()
# Create from list without schema
spark.createDataFrame(data_row).show()
# Create from list with schema
spark.createDataFrame(data_row, schema).show()
# Create from Row with columns
spark.createDataFrame(data_row, columns).show()
# Create from Row with schema
spark.createDataFrame(data_row, schema).show()
# Criar a partir de dicionário
spark.createDataFrame(data_dict).show()

+--------+-----------+
|language|users_count|
+--------+-----------+
|    Java|      20000|
|  Python|     100000|
|   Scala|       3000|
+--------+-----------+

+--------+-----------+
|language|users_count|
+--------+-----------+
|    Java|      20000|
|  Python|     100000|
|   Scala|       3000|
+--------+-----------+

+--------+-----------+
|language|users_count|
+--------+-----------+
|    Java|      20000|
|  Python|     100000|
|   Scala|       3000|
+--------+-----------+

+--------+-----------+
|language|users_count|
+--------+-----------+
|    Java|      20000|
|  Python|     100000|
|   Scala|       3000|
+--------+-----------+

+--------+-----------+
|language|users_count|
+--------+-----------+
|    Java|      20000|
|  Python|     100000|
|   Scala|       3000|
+--------+-----------+

+------+------+
|    _1|    _2|
+------+------+
|  Java| 20000|
|Python|100000|
| Scala|  3000|
+------+------+

+--------+-----------+
|language|users_count|
+--------+-----------+
|    Jav

In [78]:
structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df_names = spark.createDataFrame(data=structureData,schema=structureSchema)
df_names.printSchema()
df_names.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



+--------------------+-----+------+------+
|name                |id   |gender|salary|
+--------------------+-----+------+------+
|{James, , Smith}    |36636|M     |3100  |
|{Michael, Rose, }   |40288|M     |4300  |
|{Robert, , Williams}|42114|M     |1400  |
|{Maria, Anne, Jones}|39192|F     |5500  |
|{Jen, Mary, Brown}  |     |F     |-1    |
+--------------------+-----+------+------+



In [233]:
# write
df_customers.write.saveAsTable('table_temp_customers')
df_customers.write.format('parquet').mode('overwrite').save("../dataset/parquet_data/test.parquet")

In [16]:
# crossJoin, withColumn, withColumnRenamed
df_x = spark.range(10).withColumnRenamed('id', 'c_x')
df_y = spark.range(10).withColumnRenamed('id', 'c_y')
df_result = df_x.crossJoin(df_y).withColumn('result', col('c_x') * col('c_y'))
df_result.show()

In [35]:
# crossTab, orderBy, distinct
(df_customers
     .select('customer_state', 'customer_city')
     .distinct()
     .crosstab('customer_state', 'customer_state')
     .orderBy(col('customer_state_customer_state'))
     .show())

+-----------------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|customer_state_customer_state| AC| AL| AM| AP| BA| CE| DF| ES| GO| MA| MG| MS| MT| PA| PB| PE| PI| PR| RJ| RN| RO| RR| RS| SC| SE| SP| TO|
+-----------------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|                           AC|  8|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|                           AL|  0| 68|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|                           AM|  0|  0|  5|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|                           AP|  0|  0|  0|  6|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|                   

In [41]:
# describe
df_order_items.describe('price').show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            112650|
|   mean|120.65373901464174|
| stddev| 183.6339280502595|
|    min|              0.85|
|    max|            6735.0|
+-------+------------------+



In [44]:
# drop
df_order_items.show(2)
df_order_items.drop('freight_value').show(2)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|        19.93|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
only showing top 2 rows

+--------------------+-------------+--------------------+--------------------+-------------------+-----+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|
+--------------------+-------------+---------------

In [65]:
# dropna > na.drop
(df_customers
    .groupBy('customer_state')
    .count()
    .orderBy('customer_state')
    .replace('AL', None)
    .dropna()
    .show(3))
(df_customers
    .groupBy('customer_state')
    .count()
    .orderBy('customer_state')
    .replace('AL', None)
    .dropna(subset=['count'])
    .show(3))
(df_customers
    .groupBy('customer_state')
    .count()
    .orderBy('customer_state')
    .replace('AL', None)
    .dropna(how='all')
    .show(3))
(df_customers
    .groupBy('customer_state')
    .count()
    .orderBy('customer_state')
    .replace('AL', None)
    .dropna(thresh=1)
    .show(3))
(df_customers
    .groupBy('customer_state')
    .count()
    .orderBy('customer_state')
    .replace('AL', None)
    .dropna(thresh=2)
    .show(3))

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            AC|   81|
|            AM|  148|
|            AP|   68|
+--------------+-----+
only showing top 3 rows

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            AC|   81|
|          null|  413|
|            AM|  148|
+--------------+-----+
only showing top 3 rows

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            AC|   81|
|          null|  413|
|            AM|  148|
+--------------+-----+
only showing top 3 rows

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            AC|   81|
|          null|  413|
|            AM|  148|
+--------------+-----+
only showing top 3 rows

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            AC|   81|
|            AM|  148|
|            AP|   68|
+--------------+-----+
only showing top 3 rows



In [75]:
# fillna > na.fill
(df_customers
    .groupBy('customer_state')
    .count()
    .orderBy('customer_state')
    .replace('AL', None)
    .fillna('Unknown')
    .show(3))
(df_customers
    .groupBy('customer_state')
    .count()
    .orderBy('customer_state')
    .replace('AL', None)
    .fillna('Unknown', ['count'])
    .show(3))

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            AC|   81|
|       Unknown|  413|
|            AM|  148|
+--------------+-----+
only showing top 3 rows

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            AC|   81|
|          null|  413|
|            AM|  148|
+--------------+-----+
only showing top 3 rows



In [71]:
# Explain
(df_customers
    .groupBy('customer_state')
    .count()
    .orderBy('customer_state')
    .replace('AL', None)
    .dropna(thresh=2)
    .explain(mode='formatted'))

== Physical Plan ==
AdaptiveSparkPlan (9)
+- Project (8)
   +- Sort (7)
      +- Exchange (6)
         +- Filter (5)
            +- HashAggregate (4)
               +- Exchange (3)
                  +- HashAggregate (2)
                     +- Scan csv  (1)


(1) Scan csv 
Output [1]: [customer_state#105]
Batched: false
Location: InMemoryFileIndex [file:/Users/gabrielbossardi/Documents/Projects/pyspark-overview/dataset/csv_data/olist_customers_dataset.csv]
ReadSchema: struct<customer_state:string>

(2) HashAggregate
Input [1]: [customer_state#105]
Keys [1]: [customer_state#105]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#8394L]
Results [2]: [customer_state#105, count#8395L]

(3) Exchange
Input [2]: [customer_state#105, count#8395L]
Arguments: hashpartitioning(customer_state#105, 200), ENSURE_REQUIREMENTS, [plan_id=4739]

(4) HashAggregate
Input [2]: [customer_state#105, count#8395L]
Keys [1]: [customer_state#105]
Functions [1]: [count(1)]
Aggregate Attributes [1]

In [88]:
# join
(df_orders.join(df_customers, df_orders.customer_id == df_customers.customer_id, 'inner')
    .select(
        df_orders.order_id.alias("cupom_id"),
        df_customers.customer_state
    )
    .groupBy("customer_state").agg(count("*").alias("ct"))
    .sort(desc("ct"))
    .show(2))

(df_orders.join(df_customers, 'customer_id')
    .select(
        df_orders.order_id.alias("cupom_id"),
        df_customers.customer_state
    )
    .groupBy("customer_state").agg(count("*").alias("ct"))
    .sort(desc("ct"))
    .show(2))

+--------------+-----+
|customer_state|   ct|
+--------------+-----+
|            SP|41746|
|            RJ|12852|
+--------------+-----+
only showing top 2 rows

+--------------+-----+
|customer_state|   ct|
+--------------+-----+
|            SP|41746|
|            RJ|12852|
+--------------+-----+
only showing top 2 rows



In [94]:
# without hint
(df_orders.join(df_customers, df_orders.customer_id == df_customers.customer_id, 'inner')
    .select(
        df_orders.order_id.alias("cupom_id"),
        df_customers.customer_state
    )
    .groupBy("customer_state").agg(count("*").alias("ct"))
    .sort(desc("ct"))
    .explain(mode="formatted"))

# hint
(df_orders.join(df_customers.hint("broadcast"), df_orders.customer_id == df_customers.customer_id, 'inner')
    .select(
        df_orders.order_id.alias("cupom_id"),
        df_customers.customer_state
    )
    .groupBy("customer_state").agg(count("*").alias("ct"))
    .sort(desc("ct"))
    .explain(mode="formatted"))

== Physical Plan ==
AdaptiveSparkPlan (13)
+- Sort (12)
   +- Exchange (11)
      +- HashAggregate (10)
         +- Exchange (9)
            +- HashAggregate (8)
               +- Project (7)
                  +- BroadcastHashJoin Inner BuildLeft (6)
                     :- BroadcastExchange (3)
                     :  +- Filter (2)
                     :     +- Scan csv  (1)
                     +- Filter (5)
                        +- Scan csv  (4)


(1) Scan csv 
Output [1]: [customer_id#129]
Batched: false
Location: InMemoryFileIndex [file:/Users/gabrielbossardi/Documents/Projects/pyspark-overview/dataset/csv_data/olist_orders_dataset.csv]
PushedFilters: [IsNotNull(customer_id)]
ReadSchema: struct<customer_id:string>

(2) Filter
Input [1]: [customer_id#129]
Condition : isnotnull(customer_id#129)

(3) BroadcastExchange
Input [1]: [customer_id#129]
Arguments: HashedRelationBroadcastMode(List(input[0, string, false]),false), [plan_id=6952]

(4) Scan csv 
Output [2]: [customer_id#101, 

In [102]:
# persist
df_customers.persist(StorageLevel.MEMORY_ONLY)

DataFrame[customer_id: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string]